## Create a session

In [2]:
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *
# Create a session
Session = sessionmaker(bind=engine)
session = Session()


create database

In [ ]:
Base.metadata.create_all(engine)

In [7]:
current_file_path = "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl1/nc1/tcc/Cyclist_crosses_from_the_roadside/tcc1/tcc1.scenic"

required_behaviors = session.query(RequiredBehavior).\
                            join(RegulationRequiredBehavior).\
                            join(Regulation).\
                            join(RegulationTestCase).\
                            join(TestCase).\
                            join(TCEnhancedConcreteScenario).\
                            filter(TCEnhancedConcreteScenario.path == current_file_path).all()
for required_behavior in required_behaviors:
    print(required_behavior.behavior_name)

ego_speed_limit
ego_lane_keeping
ego_reach_destination
collision


In [ ]:
import os,re

def extract_number(filename):
    match = re.search(r'(\d+)\.scenic', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')

nl = list()
path = "/home/weidong/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/"
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        # find nls
        if re.compile(r"tcc\d+\.scenic").match(filename) and "Cyclist_crosses_from_the_roadside" in dirpath:
            nl_path = os.path.join(dirpath, filename)
            nl.append(nl_path)

nl.sort(key=extract_number)
nl

In [ ]:
import json 

with open('regulations.json') as f:
    # Load JSON data
    data = json.load(f)
    regulations = data['regulations']
    regus = [(r['regulation'], r['required_behavior']) for r in data['regulations']]
    rbhbs = [(r['required_behavior'],r['hazardous_behavior']) for r in data['regulations']]

rb_hb = dict()
for rbhb_tuple in rbhbs:
    for idx, rb in enumerate(rbhb_tuple[0]):
        if rb in rb_hb:
            if rb_hb[rb] != rbhb_tuple[1][idx]:
                print(rb)
        rb_hb[rb] =  rbhb_tuple[1][idx]



In [15]:

from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *
import os,re


# Create a session
Session = sessionmaker(bind=engine)
session = Session()
rb_sql = [RequiredBehavior(behavior_name=rb) for rb in rb_hb.keys()]
rb_hb

{'ego_speed_limit': 'Ego exceeded the applicable speed limit.',
 'ego_safe_deceleration': 'Ego performed unintended harsh brake.',
 'ego_lane_keeping': 'Ego performed unintended lane change.',
 'ego_reach_destination': 'Ego failed to reach the specified destination.',
 'collision': 'Ego collided with road user.',
 'ego_overtake_safe_lateral_distance': 'Ego did not maintain sufficient lateral distance from other road users while overtaking.',
 'ego_overtake': 'Ego performed unintended overtake .',
 'safe_longitudinal_distance': 'Ego failed to maintain the minimum required following distance from the lead vehicle.',
 'ego_safe_overtake_oncoming_traffic': 'Ego attempted to overtake despite the potential obstruction of oncoming traffic.',
 'ego_safe_overtake_higher_speed': 'Ego overtook without considerably higher speed than the target vehicle.',
 'ego_overtaking_on_the_left': 'Ego overtook on the right side.',
 'ego_being_overtaken': 'Ego increased its speed while being overtaken.'}

### Add nominal scenarios

In [ ]:
# 
functional_nominal_scenario = {}

functional_nominal_scenario_1 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/functional_scenario_description.md")
# 
logical_nominal_scenario_1 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/logical_scenario_1/logical_scenario_1.scenic")
# 
concrete_nominal_scenario_1 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
# 
functional_nominal_scenario_1.logical_nominal_scenario_items.append(logical_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_1)
# 
session.add(functional_nominal_scenario_1)
session.commit()

In [ ]:
# Nominal
functional_nominal_scenario_1 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/functional_scenario_description.md")
# 
logical_nominal_scenario_1 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/logical_scenario_1.py")
# 
concrete_nominal_scenario_1 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
concrete_nominal_scenario_2 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_2/concrete_scenario_2.py")
# 
functional_nominal_scenario_1.logical_nominal_scenario_items.append(logical_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_2)
# 
session.add(functional_nominal_scenario_1)
session.commit()

Add TCs

In [ ]:
TCs = [
    {'TC':'Parking vehicle on the roadside with opening door', 'FI':'Information processing-lidar-based object classification, Plan-behavior prediction'},
    {'TC':'E-scooter drives on motor lane', 'FI':'Plan-situation understanding'},
    {'TC':'Vehicle from the right merging lane without right of way brakes too late', 'FI':'Plan-prediction'},
    {'TC':'Workers unload a truck on the road', 'FI':'Information processing-object detection, Plan-trajectory prediction'},
    {'TC':'Pedestrian halts sharply in front of the crosswalk', 'FI':'Plan-trajectory prediction'},
    {'TC':'Front cyclist leaves the lane', 'FI':'Information processing - object detection, Plan - situation understanding'},
    {'TC':'Multiple vehicles cut in to ego path in a short time', 'FI':'Plan - situation understanding & decision making'},
    {'TC':'Front vehicle drives on lane markings', 'FI':'Plan-situation understanding'},
    {'TC':'Many vehicles park along the road side', 'FI':'Information access - offline map backup, Information processing - localization, road mark detection, Plan-situation understanding, prediction'},
    {'TC':'Traffic jam in front when neighbor lane is free', 'FI':'Plan - situation understanding + prediction'},
    {'TC':'Front vehicle halts on the lane with warning flasher on', 'FI':'Plan - situation understanding'},
    {'TC':'Cyclist crosses from the roadside', 'FI':'Information reception - FOV,, Plan - Prediction'},
    {'TC':'Oncoming vehicle overtakes a halting vehicle', 'FI':'Plan-situation understanding'},
    {'TC':'Scooter/cyclist drives on opposite lane', 'FI':'Plan-situation understanding'},
    {'TC':'Traffic jam on the opposite lane', 'FI':'Information processing-object separation, Plan-situation understanding, prediction'},
    {'TC':'Halting vehicles on both directions narrow the road', 'FI':'Plan-situation understanding'},
    {'TC':'Front cyclist moves towards the road edge and halt', 'FI':'Plan-behavior interpretation & prediction'},
]


for TC in TCs:
    session.add(TriggeringCondition(triggering_condition_name=TC['TC'], functional_insufficiency=TC['FI'], functional_insufficiency_origin='expert'))
session.commit()

### Add TC scenarios

In [ ]:
tc_enhanced_functional_scenario_1 = TCEnhancedFunctionalScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/functional_scenario_description.md",
                                                                 exist_non_complient = 0,
                                                                 functional_nominal_scenario_id = 1,
                                                                 triggering_condition_id = 12
                                                                 )
# 
tc_enhanced_logical_scenario_1 = TCEnhancedLogicalScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/logical_scenario_1/logical_scenario_1.scenic",
                                                           logical_nominal_scenario_id = 1)

tc_enhanced_functional_scenario_1.tc_enhanced_logical_scenario_items.append(tc_enhanced_logical_scenario_1)

session.add(tc_enhanced_functional_scenario_1)
session.commit()

In [ ]:


tc_enhanced_concrete_scenario_1 = TCEnhancedConcreteScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py",
                                                             concrete_nominal_scenario_id = 1,
                                                             tc_enhanced_logical_scenario_id = 1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Add regulations

In [ ]:
regulations = [
("The AD-Function shall adjust the speed of the AD-Vehicle to applicable speed limits.",("ego_speed_limit")),
("the AD-Function shall not perform harsh brake without a compelling reason.", ("ego_safe_deceleration")),
("The AD-Function shall only initiate a regular lane change if there is a reason for a lane change.", ("ego_lane_keeping")),
("The AD-Function shall reach the specified destination, in the absence of any accident.",("ego_reach_destination")),
("Under urban and rural driving conditions, the AD-Function shall avoid a collision, up to a speed of 60 km/h, with an unobstructed cyclist crossing with a lateral speed component of not more than 15 km/h in front of the vehicle. This shall be ensured independently from the specific manoeuvre the AD-Function is undertaking.", 
("collision")),
("In general, the AD-Function shall drive normally.",
("ego_safe_deceleration")),
("When overtaking pedestrians, cyclists or personal light electric vehicles in built-up areas, the AD-Function shall keep at least 1.5 m lateral distance. If necessary, the AD-Function shall wait. At intersections and junctions, the minimum lateral distance of 1.5m to cyclists does not apply, if cyclists have overtaken the AD-Vehicle waiting there on the right or have come to a standstill next to them. ",
("ego_overtake_safe_lateral_distance")),
("The AD-Function shall not overtake in unclear traffic situations.",("ego_overtake")),
("While the AD-Vehicle is not at standstill, and operating in speed range up to 60 km/h, the AD-Function shall adapt the speed to adjust the distance to a lead vehicle in the same lane to be equal or greater than the minimum following distance.",
("safe_longitudinal_distance")),
("Only if the AD-Function can oversee, that there will be no obstruction of oncoming traffic during the entire overtaking process, the AD-Function may overtake.",
("ego_safe_overtake_oncoming_traffic"))
("Only if the AD-Function drives with considerably higher speed than the vehicle being overtaken, the AD-Function may overtake.",("ego_safe_overtake_higher_speed")),
("The AD-Function shall overtake on the left, except defined otherwise in separate requirements.",("ego_overtaking_on_the_left")),
("During overtaking, AD-Function shall keep enough lateral distance to other road users.", ("ego_overtake_safe_lateral_distance")),
("When being overtaken, the AD-Function shall not increase speed.",("ego_being_overtaken")),
("In \"CCFhol: Car-to-Car Front Head-On Lane change\" scenarios, the AD-Function shall reduce speed by at least 20 km/h (original speed vs. impact speed), in case a collision cannot be avoided.",
("ego_emergency_braking")),
("If, according to traffic regulations, the AD-Vehicle is allowed to proceed or otherwise has priority, the AD-Function shall give up this right if required by the traffic situation.",
("ego_react_to_pedetrian")),              
]

for regu in regulations:
    regulation = Regulation(regulation_description=regu[0])

    tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Add required behaviors

In [ ]:

requiredNhazard_behaviors = [("collision","Ego caused collision with target."),
                      ("exceed_acceleration","Ego exceeded specified acceleration."),
                      ("minimum_distance","Ego violated minimum distance with target."),
                      ("ego_overtake","Ego overtook target unsafely."),
                      ("safe_longitudinal_distance", "Ego violated safe longitudinal distance with front vehicle."),
                      ("ego_drive_normally", "Ego drive abnormally."),
                      ("ego_overtake_safe_lateral_distance", "Ego violated safe lateral distance while overtaking target."),
                      ("ego_safe_overtake_oncoming_traffic", "Ego conduct unsafe overtake despite oncoming traffic."),
                      ("ego_safe_overtake_higher_speed", "Ego conduct unsafe overtake despite without considerably higher speed than target."),
                      ("ego_being_overtaken", "Ego increased speed while being overtaken."),
                      ("ego_emergency_braking", "Ego failed at emergency braking."),
                      ("ego_overtaking_on_the_left", "Ego overtook on the right."),
                      ("ego_react_to_pedetrian", "Ego failed at stop in front of pedestrian."),
                      ("ego_safe_deceleration","Ego deceleration was lower than threshold."),
                      ("ego_lane_keeping", "Ego deviated from its original lane."),
                      ("ego_reach_destination", "Ego failed to reach destination"),
                      ("ego_speed_limit", "Ego's speed exceeded the speed limit.")
                    ]

for behavior in requiredNhazard_behaviors[-1:]:
    required_behavior = RequiredBehavior(behavior_name=behavior[0])
    hazardous_behavior = HazardousBehavior(behavior_description=behavior[1])
    required_behavior.hazardous_behavior = hazardous_behavior

    session.add(required_behavior)

session.commit()

### Add test cases

In [ ]:
test_case_1 = TestCase()
tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Try querying

In [ ]:
# Query for a specific FunctionalNominalScenario item (replace 'functional_nominal_scenario_id' with the actual ID of the FunctionalNominalScenario item)
functional_nominal_scenario_item = session.query(FunctionalNominalScenario).where(FunctionalNominalScenario.id == 2).first()

# If functional_nominal_scenario_item is found, traverse the relationship to ConcreteNominalScenario items
if functional_nominal_scenario_item:
    # Query ConcreteNominalScenario items related to the specific FunctionalNominalScenario item
    concrete_nominal_scenario_items = session.query(ConcreteNominalScenario).join(LogicalNominalScenario).join(FunctionalNominalScenario).filter(FunctionalNominalScenario.id == functional_nominal_scenario_item.id).all()

    # Now you have the ConcreteNominalScenario items related to the specific FunctionalNominalScenario item
    for concrete_nominal_scenario_item in concrete_nominal_scenario_items:
        print(concrete_nominal_scenario_item.path)
else:
    print("FunctionalNominalScenario item not found.")

In [ ]:

required_behavior_1 = RequiredBehavior(behavior_name="ego_drive_normally", behavior_description="Ego shall drive normally without harsh brake, keep same lane, reach destination at the end.")
required_behavior_2 = RequiredBehavior(behavior_name="ego_overtake_safe_lateral_distance", behavior_description="Ego shall keep safe lateral distance when overtaking road users.")
required_behavior_3 = RequiredBehavior(behavior_name="ego_safe_overtake_oncoming_traffic", behavior_description="Ego shall only overtake when time for manuevor is not enough, given that opposite lane has oncoming traffic.")
required_behavior_4 = RequiredBehavior(behavior_name="safe_longitudinal_distance", behavior_description="Ego shall keep safe longitudinal distance.")
required_behavior_5 = RequiredBehavior(behavior_name="ego_overtake", behavior_description="ego shall not overtake.")

hazardous_behavior_1 = HazardousBehavior(behavior_description="Ego caused collision.")
hazardous_behavior_2 = HazardousBehavior(behavior_description="Ego exceeded rated acceleration.")
hazardous_behavior_3 = HazardousBehavior(behavior_description="Ego violated minimum distance.")
hazardous_behavior_4 = HazardousBehavior(behavior_description="Ego violated safe lateral distance.")
hazardous_behavior_5 = HazardousBehavior(behavior_description="Ego overtaked unsafely.")

required_behavior_1.hazardous_behavior = hazardous_behavior_1
required_behavior_2.hazardous_behavior = hazardous_behavior_2
required_behavior_3.hazardous_behavior = hazardous_behavior_3
required_behavior_4.hazardous_behavior = hazardous_behavior_4
required_behavior_5.hazardous_behavior = hazardous_behavior_5

session.add(required_behavior_1)
session.add(required_behavior_2)
session.add(required_behavior_3)
session.add(required_behavior_4)
session.add(required_behavior_5)
session.commit()

In [ ]:
# Nominal
functional_nominal_scenario_2 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/functional_scenario_description.md")
# 
logical_nominal_scenario_2 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/logical_scenario_1.py")
# 
concrete_nominal_scenario_2 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
# 
functional_nominal_scenario_2.logical_nominal_scenario_items.append(logical_nominal_scenario_2)
logical_nominal_scenario_2.concrete_nominal_scenario_items.append(concrete_nominal_scenario_2)
# 
session.add(functional_nominal_scenario_2)
session.commit()

# TC enhanced
# /home/weidong/Tools/Scenic/scenic_projects/tc_scenario_catalog/Cyclist_drives_on_opposite_lane/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py
tc_enhanced_functional_scenario_2 = TCEnhancedFunctionalScenario(path = "tc_scenario_catalog/Cyclist_drives_on_opposite_lane/functional_scenario_description.md",
                                                                 exist_non_complient = 0,
                                                                 functional_nominal_scenario_id = 2,
                                                                 triggering_condition_id = 14
                                                                 )
tc_enhanced_logical_scenario_2 = TCEnhancedLogicalScenario(path = "tc_scenario_catalog/Cyclist_drives_on_opposite_lane/logical_scenario_1/logical_scenario_1.py",
                                                           logical_nominal_scenario_id = 2)

tc_enhanced_functional_scenario_2.tc_enhanced_logical_scenario_items.append(tc_enhanced_logical_scenario_2)

session.add(tc_enhanced_functional_scenario_2)
session.commit()

# Test case
test_case_1 = TestCase()
tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()